# Project Code

Run the cell below to remove old save files

In [ ]:
import os
os.remove("/kaggle/working/roberta_phishing_model_metrics.pkl")
os.remove("/kaggle/working/state.db")
os.remove("/kaggle/working/roberta_phishing_model_fold_1.pkl")
os.remove("/kaggle/working/roberta_phishing_model_fold_2.pkl")
os.remove("/kaggle/working/roberta_phishing_model_fold_3.pkl")
os.remove("/kaggle/working/roberta_phishing_model_fold_4.pkl")
os.remove("/kaggle/working/roberta_phishing_model_fold_5.pkl")

## Part 1: 5 Fold CV

**Dataset Loading and Preparation**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, precision_recall_curve, roc_curve, auc, f1_score, confusion_matrix, precision_score, recall_score
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import AdamW, get_scheduler
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.utils.data import RandomSampler, SequentialSampler
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import roc_auc_score
import pickle


# Load the dataset
dataset = pd.read_csv("/kaggle/input/phishing-urls/phishing_site_urls.csv").sample(n=50000, random_state=42)

# Data Preprocessing and remove missing values
dataset = dataset.dropna()

# Map 'good' to 0 and 'bad' to 1
dataset['Label'] = dataset['Label'].map({'good': 0, 'bad': 1})

# Tokenizer and model initialization
model_name = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name,local_files_only=False)
model = RobertaForSequenceClassification.from_pretrained(model_name,local_files_only=False)

# Cross-validation setup
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize variables for loss
steps = []
losses = []

# Parallelize over available GPUs
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
if torch.cuda.device_count() > 1:
    print("Using", torch.cuda.device_count(), "GPUs!")
    model = torch.nn.DataParallel(model)


**Cross Validation Loop**

In [ ]:
all_y_true=[]
all_y_pred=[]
for fold, (train_idx, val_idx) in enumerate(skf.split(dataset['URL'], dataset['Label'])):
    print(f"\nFold {fold + 1}/{skf.n_splits}")
    model_name = "roberta-base"
    tokenizer = RobertaTokenizer.from_pretrained(model_name,local_files_only=False)
    model = RobertaForSequenceClassification.from_pretrained(model_name,local_files_only=False)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    if torch.cuda.device_count() > 1:
        print("Using", torch.cuda.device_count(), "GPUs!")
        model = torch.nn.DataParallel(model)

    # Split the dataset into training and validation sets
    train_data = dataset.iloc[train_idx]
    val_data = dataset.iloc[val_idx]

    # Tokenize and encode the input text for training and validation
    train_encodings = tokenizer(list(train_data['URL']), truncation=True, padding=True, max_length=512, return_tensors='pt', return_attention_mask=True)
    val_encodings = tokenizer(list(val_data['URL']), truncation=True, padding=True, max_length=512, return_tensors='pt', return_attention_mask=True)

    # Create DataLoader for training and validation data
    train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], torch.tensor(list(train_data['Label'])))
    val_dataset = TensorDataset(val_encodings['input_ids'], val_encodings['attention_mask'], torch.tensor(list(val_data['Label'])))

    train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=32)
    val_dataloader = DataLoader(val_dataset, sampler=SequentialSampler(val_dataset), batch_size=64)

    optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=1e-3)

    # Fine-tuning
    epochs = 4
    total_steps = len(train_dataloader) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=int(0.1 * total_steps), num_training_steps=total_steps)
    loss_fn = torch.nn.CrossEntropyLoss()

    # Model training loop
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        model.train()
        progress_bar = tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc=f'Epoch {epoch + 1}/{epochs}', leave=False)
        for step, batch in progress_bar:
            train_input_ids, train_attention_mask, labels = batch
            train_input_ids = train_input_ids.to(device)
            train_attention_mask = train_attention_mask.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(train_input_ids, attention_mask=train_attention_mask, labels=labels)
            loss = outputs.loss.mean()
            #loss = outputs.loss
            loss.backward()
            optimizer.step()
            scheduler.step()

            # Append loss values
            steps.append(len(steps) + 1)
            losses.append(loss.item())
        
    model.eval()
    val_losses = []
    val_predictions = []
    val_true_labels = []
    with torch.no_grad():
        for val_batch in val_dataloader:
            val_input_ids, val_attention_mask, val_labels = val_batch
            val_input_ids = val_input_ids.to(device)
            val_attention_mask = val_attention_mask.to(device)
            val_labels = val_labels.to(device)

            val_outputs = model(val_input_ids, attention_mask=val_attention_mask, labels=val_labels)
            val_loss = val_outputs.loss.mean().item()
            val_predictions.extend(torch.argmax(val_outputs.logits, dim=1).cpu().numpy())
            val_true_labels.extend(val_labels.cpu().numpy())
            val_losses.append(val_loss)

    # Calculate validation metrics
    val_accuracy = accuracy_score(val_true_labels, val_predictions)
    val_f1 = f1_score(val_true_labels, val_predictions, average='weighted')
    val_roc_auc = roc_auc_score(val_true_labels, val_predictions)
    val_precision = precision_score(val_true_labels, val_predictions, average='weighted')
    val_recall = recall_score(val_true_labels, val_predictions, average='weighted')
    val_classification_report = classification_report(val_true_labels, val_predictions)

    print(f"Validation Loss: {np.mean(val_losses):.4f}, Accuracy: {val_accuracy:.4f}, F1 Score: {val_f1:.4f}, ROC AUC: {val_roc_auc:.4f}")
    print(f"Precision: {val_precision:.4f}, Recall: {val_recall:.4f}")
    print("Classification Report:\n", val_classification_report)
    # Append predictions and true labels for each fold
    all_y_true.extend(val_true_labels)
    all_y_pred.extend(val_predictions)
    

    # Save the trained model for each fold
    model_save_path = f"roberta_phishing_model_fold_{fold + 1}.pkl"
    model_state = {
        'model': model.state_dict(),
        'tokenizer': tokenizer,
        'train_dataloader': train_dataloader,
        'val_dataloader': val_dataloader,
        'val_true_labels': val_true_labels,
        'val_predictions': val_predictions,
        'val_f1': val_f1,
        'val_roc': val_roc_auc,
        'val_prec': val_precision,
        'val_recall': val_recall,
        'val_classification':val_classification_report
    }
    with open(model_save_path, 'wb') as model_file:
        pickle.dump(model_state, model_file)
        
# Save the overall model predictions
result_save_path = f"roberta_phishing_model_metrics.pkl"
metrics = {
    'y_true': all_y_true,
    'y_pred': all_y_pred
}
with open(result_save_path, 'wb') as model_file:
    pickle.dump(metrics, model_file)

# Calculate overall metrics after all folds
accuracy = accuracy_score(all_y_true, all_y_pred)
classification_rep = classification_report(all_y_true, all_y_pred, target_names=['good','bad'])
conf_matrix = confusion_matrix(all_y_true, all_y_pred)
precision, recall, _ = precision_recall_curve(all_y_true, all_y_pred)
fpr, tpr, _ = roc_curve(all_y_true, all_y_pred)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Class Imbalance Visualization
plt.figure(figsize=(8, 6))
sns.countplot(x='Label', data=dataset)
plt.title('Class Distribution')
plt.show()

# Precision-Recall Curve
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, marker='.')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.show()

# ROC Curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, marker='.')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()

**Print the calculated metrics**

In [ ]:
from sklearn.metrics import classification_report, accuracy_score,roc_auc_score, precision_recall_curve, roc_curve, auc, f1_score, confusion_matrix, precision_score, recall_score
import pickle
with open('/kaggle/input/metrics/roberta_phishing_model_metrics.pkl', 'rb') as model_file:
    model_state = pickle.load(model_file)
    
all_y_true = model_state['y_true']
all_y_pred = model_state['y_pred']

accuracy = accuracy_score(all_y_true, all_y_pred)
classification_rep = classification_report(all_y_true, all_y_pred, target_names=['good','bad'])
f1score = f1_score(all_y_true, all_y_pred, average='weighted')
print("AUROC: ",roc_auc_score(all_y_true, all_y_pred))
print("Accuracy: ",accuracy)
print("F1 score: ",f1score)
print(classification_rep)


## Part 2: Train/Test Split

**Without Class Weights**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, precision_recall_curve, roc_curve, auc, f1_score
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaConfig
import torch
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_scheduler
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_auc_score, roc_curve, auc
import seaborn as sns


# Load the dataset
dataset = pd.read_csv("/kaggle/input/phishing-site-urls/phishing_site_urls.csv")

# Data Preprocessing and remove missing values
dataset = dataset.dropna()

# Encode labels (good and bad) to numerical values
label_encoder = LabelEncoder()
dataset['Label'] = label_encoder.fit_transform(dataset['Label'])

# Split the dataset into training and testing sets
X = dataset['URL']
y = dataset['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.82, random_state=42)

# RoBERTa Tokenization and Feature Extraction
model_name = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name)

# Tokenize and encode the input text
train_encodings = tokenizer(list(X_train), truncation=True, padding=True, max_length=512, return_tensors='pt', return_attention_mask=True)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True, max_length=512, return_tensors='pt', return_attention_mask=True)

# Create DataLoader for training and testing data
train_dataset = torch.utils.data.TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], torch.tensor(list(y_train)))
test_dataset = torch.utils.data.TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], torch.tensor(list(y_test)))

# Parallelize over available GPUs
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
# Wrap the model with DataParallel
model = torch.nn.DataParallel(model, device_ids = [ 0, 1])
model.to(f'cuda:{model.device_ids[0]}')

# class_weights = torch.tensor([1.0, float(len(y_train[y_train == 0])) / float(len(y_train[y_train == 1]))])
# class_weights = class_weights.to(device)

train_batch_size = 32
test_batch_size = 64

train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=train_batch_size)
test_dataloader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size=test_batch_size)

optimizer = AdamW(model.parameters(), lr=1e-5)

# Fine-tuning
epochs = 6
total_steps = len(train_dataloader) * epochs
scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=int(0.1 * total_steps),
    num_training_steps=total_steps,
)

loss_fn = torch.nn.CrossEntropyLoss()
model.train()

# Initialize variables for loss
steps = []
losses = []
fig, ax = plt.subplots()

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    progress_bar = tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc=f'Epoch {epoch + 1}/{epochs}', leave=False)
    for step, batch in progress_bar:
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs['loss']
        loss = loss.mean() 
        loss.backward()
        optimizer.step()
        scheduler.step()

        # Append loss values for real-time plot
        steps.append(len(steps) + 1)
        losses.append(loss.item())

# Prediction
model.eval()
predictions = []
with torch.no_grad():
    progress_bar = tqdm(test_dataloader, desc='Evaluating', leave=False)
    for batch in progress_bar:
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        predictions.extend(torch.argmax(outputs.logits, dim=1).cpu().numpy())

# Evaluate the model
y_true = list(y_test)
y_pred = predictions
accuracy = accuracy_score(y_true, y_pred)
classification_rep = classification_report(y_true, y_pred, target_names=label_encoder.classes_)
f1 = f1_score(y_true, y_pred, average='weighted')  # Calculate F1 score

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(classification_rep)

# Visualization
# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# Class Imbalance Visualization
sns.countplot(x='Label', data=dataset)
plt.title('Class Distribution')
plt.show()

# Precision-Recall Curve
precision, recall, _ = precision_recall_curve(y_true, y_pred)
plt.plot(recall, precision, marker='.')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.show()

# ROC Curve and AUC
fpr, tpr, _ = roc_curve(y_true, y_pred)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, marker='.')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()

#saving the model
import pickle
model_save_path = "roberta_phishing_model.pkl"
model_state = {
    'model': model.state_dict(),
#     'class_weights': class_weights.cpu().numpy(),
    'test_dataloader':test_dataloader,
    'train_dataloader': train_dataloader,
    'device': device,
    'train_encodings': train_encodings,
    'test_encodings': test_encodings,
    'train_dataset': train_dataset,
    'test_dataset': test_dataset,
    'tokenizer': tokenizer
}

with open(model_save_path, 'wb') as model_file:
    pickle.dump(model_state, model_file)

# Save training-related data for later visualization
training_data_save_path = "training_data.pkl"
training_data = {
    'steps': steps,
    'losses': losses
}

with open(training_data_save_path, 'wb') as training_data_file:
    pickle.dump(training_data, training_data_file)

**With Class Weights**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, precision_recall_curve, roc_curve, auc, f1_score
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaConfig
import torch
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_scheduler
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_auc_score, roc_curve, auc
import seaborn as sns


# Load the dataset
dataset = pd.read_csv("/kaggle/input/phishing-site-urls/phishing_site_urls.csv")

# Data Preprocessing and removeing missing values
dataset = dataset.dropna()

# Encode labels (good and bad) to numerical values
label_encoder = LabelEncoder()
dataset['Label'] = label_encoder.fit_transform(dataset['Label'])

# Split the dataset into training and testing sets
X = dataset['URL']
y = dataset['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.82, random_state=42)

# RoBERTa Tokenization and Feature Extraction
model_name = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name)

# Tokenize and encode the input text
train_encodings = tokenizer(list(X_train), truncation=True, padding=True, max_length=512, return_tensors='pt', return_attention_mask=True)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True, max_length=512, return_tensors='pt', return_attention_mask=True)

# Create DataLoader for training and testing data
train_dataset = torch.utils.data.TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], torch.tensor(list(y_train)))
test_dataset = torch.utils.data.TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], torch.tensor(list(y_test)))

# Parallelize over available GPUs
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
# Wrap the model with DataParallel
model = torch.nn.DataParallel(model, device_ids = [ 0, 1])
model.to(f'cuda:{model.device_ids[0]}')

class_weights = torch.tensor([1.0, float(len(y_train[y_train == 0])) / float(len(y_train[y_train == 1]))])
class_weights = class_weights.to(device)

train_batch_size = 32
test_batch_size = 64

train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=train_batch_size)
test_dataloader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size=test_batch_size)

optimizer = AdamW(model.parameters(), lr=1e-5)

# Fine-tuning
epochs = 6
total_steps = len(train_dataloader) * epochs
scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=int(0.1 * total_steps),
    num_training_steps=total_steps,
)

loss_fn = torch.nn.CrossEntropyLoss()
model.train()

# Initialize variables for loss
steps = []
losses = []
fig, ax = plt.subplots()

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    progress_bar = tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc=f'Epoch {epoch + 1}/{epochs}', leave=False)
    for step, batch in progress_bar:
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs['loss']
        loss = loss.mean() 
        loss.backward()
        optimizer.step()
        scheduler.step()

        # Append loss values for real-time plot
        steps.append(len(steps) + 1)
        losses.append(loss.item())

# Prediction
model.eval()
predictions = []
with torch.no_grad():
    progress_bar = tqdm(test_dataloader, desc='Evaluating', leave=False)
    for batch in progress_bar:
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        predictions.extend(torch.argmax(outputs.logits, dim=1).cpu().numpy())

# Evaluate the model
y_true = list(y_test)
y_pred = predictions
accuracy = accuracy_score(y_true, y_pred)
classification_rep = classification_report(y_true, y_pred, target_names=label_encoder.classes_)
f1 = f1_score(y_true, y_pred, average='weighted')  # Calculate F1 score

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(classification_rep)

# Visualization
# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# Class Imbalance Visualization
sns.countplot(x='Label', data=dataset)
plt.title('Class Distribution')
plt.show()

# Precision-Recall Curve
precision, recall, _ = precision_recall_curve(y_true, y_pred)
plt.plot(recall, precision, marker='.')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.show()

# ROC Curve and AUC
fpr, tpr, _ = roc_curve(y_true, y_pred)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, marker='.')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()

#saving the model
import pickle
model_save_path = "roberta_phishing_model.pkl"
model_state = {
    'model': model.state_dict(),
    'class_weights': class_weights.cpu().numpy(),
    'test_dataloader':test_dataloader,
    'train_dataloader': train_dataloader,
    'device': device,
    'train_encodings': train_encodings,
    'test_encodings': test_encodings,
    'train_dataset': train_dataset,
    'test_dataset': test_dataset,
    'tokenizer': tokenizer
}

with open(model_save_path, 'wb') as model_file:
    pickle.dump(model_state, model_file)

# Save training-related data for later visualization
training_data_save_path = "training_data.pkl"
training_data = {
    'steps': steps,
    'losses': losses
}

with open(training_data_save_path, 'wb') as training_data_file:
    pickle.dump(training_data, training_data_file)

## SHAP Values 

In [ ]:
import shap
labels = sorted(model.config.label2id, key=model.config.label2id.get)


# This defines an explicit python function that takes a list of strings and outputs scores for each class
def f(x):
    tv = torch.tensor(
        [
            tokenizer.encode(v, padding="max_length", max_length=128, truncation=True)
            for v in x
        ]
    ).cuda()
    attention_mask = (tv != 0).type(torch.int64).cuda()
    outputs = model(tv, attention_mask=attention_mask)[0].detach().cpu().numpy()
    scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
    val = sp.special.logit(scores)
    return val

In [ ]:
import scipy as sp
method = "custom tokenizer"

# build an explainer by passing a transformers tokenizer
if method == "transformers tokenizer":
    explainer = shap.Explainer(f, tokenizer, output_names=labels)

# build an explainer by explicitly creating a masker
elif method == "default masker":
    masker = shap.maskers.Text(r"\W")  # this will create a basic whitespace tokenizer
    explainer = shap.Explainer(f, masker, output_names=labels)

# build a fully custom tokenizer
elif method == "custom tokenizer":
    import re

    def custom_tokenizer(s, return_offsets_mapping=True):
        """Custom tokenizers conform to a subset of the transformers API."""
        pos = 0
        offset_ranges = []
        input_ids = []
        for m in re.finditer(r"\W", s):
            start, end = m.span(0)
            offset_ranges.append((pos, start))
            input_ids.append(s[pos:start])
            pos = end
        if pos != len(s):
            offset_ranges.append((pos, len(s)))
            input_ids.append(s[pos:])
        out = {}
        out["input_ids"] = input_ids
        if return_offsets_mapping:
            out["offset_mapping"] = offset_ranges
        return out

    masker = shap.maskers.Text(custom_tokenizer)
    explainer = shap.Explainer(f, masker, output_names=labels)

In [ ]:
shap_values = explainer(X_train[:3])
shap.plots.text(shap_values)